### Install dependencies

In [192]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.1
    Uninstalling grpcio-1.62.1:
      Successfully uninstalled grpcio-1.62.1


### Load CSV data from S3

In [2]:
!pwd

/home/ec2-user/SageMaker/Enterprise-RAG/notebooks


In [125]:
import boto3
import pandas as pd

s3 = boto3.client('s3')

bucket_name = 'recipes-rag'
file_key = 'food_recipes.csv'

# Download the file from s3 locally
s3.download_file(bucket_name, file_key, '../data/food_recipes.csv')

# Load the CSV into a DataFrame
df = pd.read_csv('../data/food_recipes.csv')

df.head()

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,https://www.archanaskitchen.com/tomato-gotsu-r...,good,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,20 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,good,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,0 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,good,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,20 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,good,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,30 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes


### Load data into chunked documents

In [126]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [190]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-03 05:42:43.592498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one ha

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [128]:
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)

In [129]:
def create_documents(df):
    # AWS Kendra requires non empty string values for each field
    # Make explicit to the LLM that the field is not available
    df.fillna('Not Available', inplace=True)
    
    documents = []
    for index, row in df.iterrows():
        metadata = {
            'recipe_title': str(row['recipe_title']) if row['recipe_title'] else 'No Title Available',
            'url': str(row['url']) if row['url'] else 'https://example.com',
            # 'record_health': str(row['record_health']) if row['record_health'] else 'Unknown',
            'vote_count': str(row['vote_count']) if row['vote_count'] else 'No Votes Available',
            'rating': str(row['rating']) if row['rating'] else 'No Rating Available',
            'cuisine': str(row['cuisine']) if row['cuisine'] else 'No Cuisine Available',
            'course': str(row['course']) if row['course'] else 'No Course Available',
            'diet': str(row['diet']) if row['diet'] else 'No Diet Information Available',
            'prep_time': str(row['prep_time']) if row['prep_time'] else 'No Prep Time Available',
            'cook_time': str(row['cook_time']) if row['cook_time'] else 'No Cook Time Available',
            'author': str(row['author']) if row['author'] else 'No Author Available',
            'category': str(row['category']) if row['category'] else 'No Category Available'
        }

        # Combine all text fields for the document content
        text = f"{row['description']} {row['ingredients']} {row['instructions']} {row['tags']}"
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)
    
    return documents

In [130]:
documents = create_documents(df)

In [131]:
documents[0]

Document(metadata={'recipe_title': 'Roasted Peppers And Mushroom Tortilla Pizza Recipe', 'url': 'https://www.archanaskitchen.com/roasted-peppers-and-mushroom-tortilla-pizza-recipe', 'vote_count': '434', 'rating': '4.9585253456221', 'cuisine': 'Mexican', 'course': 'Dinner', 'diet': 'Vegetarian', 'prep_time': '15 M', 'cook_time': '15 M', 'author': 'Divya Shivaraman ', 'category': 'Pizza Recipes'}, page_content=' is a quicker version pizza to satisfy your cravings. It is a very quick and easy recipe for days that you do not feel like cooking a full fledged meal. With the preference of toppings of your choice this pizza recipe is definitely a winner at any home. The toppings used in this  has some roasted peppers, mushroom with loaded cheese and marinara sauce. Enjoy this easy recipe with your favorite toppings.\xa0 This is a great recipe, if you are looking for an Indian/Fusion Pizza or a Homemade Pizza recipe. Serve  along with  \xa0and   for a weekend night dinner. If you like this reci

In [132]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [133]:
split_documents = split_documents_with_metadata(documents, text_splitter_recursive)

### Init kendra vector db and add documents

In [124]:
from langchain.retrievers import AmazonKendraRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [12]:
aws_region = "us-east-1"
kendra_index_id = "b912ca91-f69c-49b1-8e40-4120ef57753d"

In [13]:
# Initialize Kendra client
kendra_client = boto3.client('kendra', aws_region)

In [177]:
# Function to list all documents in Kendra index
def get_all_document_ids_in_kendra(show_output=False):
    document_ids = []
    response = kendra_client.query(
            IndexId = kendra_index_id,
            AttributeFilter = {
                "EqualsTo": {      
                    "Key": "_language_code",
                    "Value": {
                        "StringValue": "en"
                        }
                    }
                },
            SortingConfiguration = {
                "DocumentAttributeKey": "_created_at",
                "SortOrder": "DESC"})
    
    if show_output:
        print(response)
    
    for result in response['ResultItems']:
        if result['Type'] == 'DOCUMENT':
            document_ids.append(result['DocumentId'])
    
    return document_ids


In [179]:
# Function to delete documents from Kendra index
def delete_documents_from_kendra():
    document_ids = get_all_document_ids_in_kendra()
    for doc_id in document_ids:
        kendra_client.batch_delete_document(
            IndexId=kendra_index_id,
            DocumentIdList=[doc_id]
        )
    print(f"Deleted {len(document_ids)} documents from Kendra index.")

In [139]:
def clear_kendra_metadata():
    print("Clearing existing Kendra index metadata configuration...")
    response = kendra_client.update_index(
        Id=kendra_index_id,
        DocumentMetadataConfigurationUpdates=[]
    )
    print("Cleared Kendra index metadata configuration.")

In [142]:
# ==================== NOTE: THIS FUNCTION WIPES THE DB CLEAN AND THE EXISTING METADATA CONFIG ====================
# update the function with new metadata as it changes
def update_kendra_metadata():
    delete_documents_from_kendra()
    clear_kendra_metadata()
    
    print("Updating Kendra index metadata configuration...")
    response = kendra_client.update_index(
        Id=kendra_index_id,
        DocumentMetadataConfigurationUpdates=[
            # {
            #     'Name': 'record_health',
            #     'Type': 'STRING_VALUE'
            # },
            {
                'Name': 'vote_count',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'rating',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'cuisine',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'course',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'diet',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'prep_time',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'cook_time',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'author',
                'Type': 'STRING_VALUE'
            },
            {
                'Name': 'category',
                'Type': 'STRING_VALUE'
            }
        ]
    )
    print("Updated Kendra index metadata configuration.")

In [186]:
import time

# Function to add documents to db
# ==================== NOTE: THIS FUNCTION WIPES THE DB CLEAN BEFORE ADDING THE NEW DOCUMENTS ====================
# Kendra allows only max batch size of 10
# THIS TAKES ABOUT ~15 MIN TO RUN
def add_documents_to_kendra(docs, batch_size=10, delay=1):
    delete_documents_from_kendra()
    
    document_ids = []
    total_batches = len(docs) // batch_size + (1 if len(docs) % batch_size > 0 else 0)
    
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        document_list = []
        for doc in batch:
            document = {
                'Id': doc.metadata['url'],
                'Title': doc.metadata['recipe_title'],
                'Blob': doc.page_content.encode('utf-8'),
                'ContentType': 'PLAIN_TEXT',
                'Attributes': [
                    # {'Key': 'record_health', 'Value': {'StringValue': doc.metadata['record_health']}},
                    {'Key': 'vote_count', 'Value': {'StringValue': doc.metadata['vote_count']}},
                    {'Key': 'rating', 'Value': {'StringValue': doc.metadata['rating']}},
                    {'Key': 'cuisine', 'Value': {'StringValue': doc.metadata['cuisine']}},
                    {'Key': 'course', 'Value': {'StringValue': doc.metadata['course']}},
                    {'Key': 'diet', 'Value': {'StringValue': doc.metadata['diet']}},
                    {'Key': 'prep_time', 'Value': {'StringValue': doc.metadata['prep_time']}},
                    {'Key': 'cook_time', 'Value': {'StringValue': doc.metadata['cook_time']}},
                    {'Key': 'author', 'Value': {'StringValue': doc.metadata['author']}},
                    {'Key': 'category', 'Value': {'StringValue': doc.metadata['category']}}
                ]
            }
            document_list.append(document)
        
        try:
            response = kendra_client.batch_put_document(
                IndexId=kendra_index_id,
                Documents=document_list
            )
            document_ids.extend([doc['Id'] for doc in document_list])
            if 'FailedDocuments' in response and response['FailedDocuments']:
                for failure in response['FailedDocuments']:
                    print(f"Failed to add document {failure['Id']}: {failure['ErrorCode']} - {failure['ErrorMessage']}")
        except kendra_client.exceptions.ThrottlingException:
            print(f"ThrottlingException encountered. Retrying batch starting from document {i}.")
            time.sleep(delay)
            response = kendra_client.batch_put_document(
                IndexId=kendra_index_id,
                Documents=document_list
            )
            document_ids.extend([doc['Id'] for doc in document_list])
            if 'FailedDocuments' in response and response['FailedDocuments']:
                for failure in response['FailedDocuments']:
                    print(f"Failed to add document {failure['Id']}: {failure['ErrorCode']} - {failure['ErrorMessage']}")

        print(f"Batch {i // batch_size + 1} of {total_batches} added successfully.")
        time.sleep(delay)  # Wait before sending the next batch
    
    return document_ids

In [187]:
# Function to check the status of documents
def check_document_status(document_ids):
    statuses = []
    for doc_id in document_ids:
        response = kendra_client.batch_get_document_status(
            IndexId=kendra_index_id,
            DocumentInfoList=[{'DocumentId': doc_id}]
        )
        statuses.append(response)
    return statuses

In [188]:
# Only call this once if metadata fields change, deletes config and all data
# update the function with new metadata
update_kendra_metadata()

Deleted 10 documents from Kendra index.
Clearing existing Kendra index metadata configuration...
Cleared Kendra index metadata configuration.
Updating Kendra index metadata configuration...
Updated Kendra index metadata configuration.


In [189]:
document_ids = add_documents_to_kendra(documents)

Deleted 10 documents from Kendra index.
Batch 1 of 801 added successfully.
Batch 2 of 801 added successfully.
Batch 3 of 801 added successfully.
Batch 4 of 801 added successfully.
Batch 5 of 801 added successfully.
Batch 6 of 801 added successfully.
Batch 7 of 801 added successfully.
Batch 8 of 801 added successfully.
Batch 9 of 801 added successfully.
Batch 10 of 801 added successfully.
Batch 11 of 801 added successfully.
Batch 12 of 801 added successfully.
Batch 13 of 801 added successfully.
Batch 14 of 801 added successfully.
Batch 15 of 801 added successfully.
Batch 16 of 801 added successfully.
Batch 17 of 801 added successfully.
Batch 18 of 801 added successfully.
Batch 19 of 801 added successfully.
Batch 20 of 801 added successfully.
Batch 21 of 801 added successfully.
Batch 22 of 801 added successfully.
Batch 23 of 801 added successfully.
Batch 24 of 801 added successfully.
Batch 25 of 801 added successfully.
Batch 26 of 801 added successfully.
Batch 27 of 801 added successfull

#### This uses in memory qdrant database so we can experiment faster without having to reinitialize the kendra database. We should progress to using a hosted solution like Kendra or self host qdrant on AWS once document chunking and ranking is in a good state.

In [193]:
# This is faster than deleting and rebuilding the kendra db
# but it still takes a few minutes to run
from langchain_community.vectorstores import Qdrant

qdrant_store = Qdrant.from_documents(split_documents,
    embedding_model,
    location=":memory:",
)

In [194]:
qdrant_retriever = qdrant_store.as_retriever()

In [195]:
# Initialize the Kendra retriever
kendra_retriever = AmazonKendraRetriever(
    index_id=kendra_index_id,
    region=aws_region
)

### Init bedrock model

In [196]:
import json

In [346]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [435]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model_with_response_stream(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    
    return response['body']

### Pipe langchain together

In [436]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [437]:
# We will need to tune these prompts

baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.
You will be provided a list of relevant context about relevant food recipes
which includes metadata like ingredients and cooking instructions.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [438]:
baseline_user_prompt = """
### Here is a context:
{context} 

### Here is a user prompt:
{query}
"""

In [439]:
def process_prompt(query_args):
    prompt_with_context = baseline_user_prompt.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [440]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

In [441]:
import sys

def parse_event_stream(stream):
    for event in stream:
        if 'chunk' in event:
            chunk_data = event['chunk']['bytes']
            chunk_json = json.loads(chunk_data.decode('utf-8'))
            
            if chunk_json['type'] == 'content_block_delta':
                delta_text = chunk_json['delta'].get('text', '')
                sys.stdout.write(delta_text)
                
            sys.stdout.flush()

In [442]:
qdrant_rag_chain = (
    RunnableMap(
        {"context": qdrant_retriever | format_docs,
         "query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_llm
    | parse_event_stream
)

In [447]:
kendra_rag_chain = (
    RunnableMap(
        {"context": kendra_retriever | format_docs,
         "query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_llm
    | parse_event_stream
)

### Try it out

In [444]:
test_query_1 = "I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions"

In [445]:
qdrant_rag_chain.invoke(test_query_1)

Based on your preference for Asian fusion food and being a vegetarian, I would recommend trying the "Chinese Tacos (Salad Taco with Hot and Sweet Vegetables) - Fusion Recipe" from the provided context. Here are the ingredients and instructions:

Ingredients:
- 1 cup shredded cabbage
- 1 cup shredded carrot
- 1 cup shredded capsicum (bell pepper)
- 1 cup shredded onion
- 2 tablespoons soy sauce
- 1 tablespoon rice vinegar
- 1 teaspoon sesame oil
- Salt and pepper to taste
- 8-10 taco shells or lettuce leaves
- For the Hot and Sweet Vegetable Filling:
  - 1 cup shredded cabbage
  - 1 cup shredded carrot
  - 1 cup shredded capsicum (bell pepper)
  - 2 tablespoons soy sauce
  - 1 tablespoon rice vinegar
  - 1 teaspoon sesame oil
  - 1 teaspoon red chili sauce
  - 1 teaspoon honey
  - Salt and pepper to taste

Instructions:

1. In a bowl, mix together the shredded cabbage, carrot, capsicum, onion, soy sauce, rice vinegar, sesame oil, salt, and pepper. Set aside.

2. For the Hot and Sweet Ve

In [448]:
kendra_rag_chain.invoke(test_query_1)

Based on your preference for vegetarian Asian fusion cuisine, I would recommend trying the Spicy Eggplant & Capsicum Curry (Indo-Chinese Fusion Recipe). Here are the ingredients and instructions:

Ingredients:
- 1 large eggplant (aubergine/brinjal), cut into 1-inch cubes
- 1 large capsicum (bell pepper), cut into 1-inch pieces
- 3 cloves garlic, finely chopped
- 1 onion, finely chopped
- 1 cup water
- 2 tbsp soy sauce
- 1-2 tsp chili garlic sauce (or to taste)
- 1 tsp vinegar
- 1 tsp sugar
- 2 tbsp cornflour (corn starch)
- 2 tbsp oil
- Salt to taste
- Chopped cilantro for garnish
- Roasted peanuts (optional) for garnish

Instructions:

1. In a bowl, mix together the water, soy sauce, chili garlic sauce, vinegar, sugar, and cornflour until well combined and no lumps remain.

2. Heat oil in a wok or pan and sauté the chopped garlic and onions until fragrant.

3. Pour the sauce mixture into the wok and add the cubed eggplant and capsicum pieces. Let it come to a boil.

4. Allow the sauce

### What docs were returned?

In [248]:
qdrant_retriever.invoke(test_query_1)

[Document(metadata={'recipe_title': 'Chinese Tacos (Salad Taco with Hot and Sweet Vegetables) -Fusion Recipe', 'url': 'https://www.archanaskitchen.com/chinese-tacos-salad-taco-with-hot-and-sweet-vegetables-fusion-recipe', 'vote_count': '504', 'rating': '4.8650793650794', 'cuisine': 'Fusion', 'course': 'Dinner', 'diet': 'Vegetarian', 'prep_time': '10 M', 'cook_time': '45 M', 'author': 'Uma Raghuraman', 'category': 'Indian Chinese Recipes', 'chunk_id': 1, '_id': 'fd04f99b7b5f4801ae40651fa023d71e', '_collection_name': '50f12f2b483140528012c54b378ec58e'}, page_content='-Fusion Recipe.|Puree the tomatoes with green chilli in a mixer without adding any water.|Heat a wok/karahi with oil over low flame. Add grated garlic.|When the garlic slightly changes colour, add the pureed tomatoes. Let the tomatoes cook over low flame till the excess water evaporates and the oil separates. This step takes up to 10 minutes. Keep stirring occasionally. Your spicy tomato sauce is ready!|Heat a vessel with 6 

In [223]:
kendra_retriever.invoke(test_query_1)

[Document(metadata={'result_id': '913b20a7-4192-463a-b270-1ee89e5f174f-2a721ef2-87a0-40f1-adf5-8bca69f85099', 'document_id': 'https://www.archanaskitchen.com/kathal-biryani-recipe-raw-jackfruit-pilaf', 'source': '', 'title': 'Kathal Biryani Recipe- Raw Jackfruit Biryani Topped with Caramelised Onions', 'excerpt': 'fried raisins and cashews-save few to garnish before serving. |Cover the vessel with aluminium foil and seal it from the sides. |Place the lid of the vessel on top of the foil and keep a heavy vessel over it so that the aroma is intact and the Kathal Biryani\xa0cooks well in dum.\xa0 Simmer the gas and let the\xa0Kathal Biryani\xa0cook on dum for 20 minutes. |Once the Kathal Biryani is done, remove the foil carefully and mix well by shaking the vessel. By doing this the yakhni mixes well with the rice without breaking the grains. By doing this the yakhni mixes well with the rice without breaking the grains.\xa0 |Serve the\xa0Kathal Biryani\xa0as a one dish meal for a weeknigh